In [1]:
import os, glob

In [3]:
data_dir = 'data/txt'
chunk_size = 256
overlap_size = 64

In [2]:
def split_text_to_chunks(text, chunk_size=1000, overlap_size=100):
    words = text.split()
    chunks = []

    for i in range(0, len(words), chunk_size - overlap_size):
        chunk = " ".join(words[i : i + chunk_size])
        chunks.append(chunk)

        if i + chunk_size + overlap_size >= len(words):
            break

    return chunks

In [ ]:
file_paths = glob.glob(os.path.join(data_dir, '*.txt'))
all_chunks_with_metadata = []

for file_path in file_paths:
    basename = os.path.basename(file_path) 

    with open(file_path, 'r') as f:
        text = f.read()

    text_chunks = split_text_to_chunks(text, chunk_size, overlap_size)

    for chunk in text_chunks:
        all_chunks_with_metadata.append({
            'text': chunk,
            'source_document': basename,
            'model_name' : os.path.splitext(basename)[0]
        })

In [9]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\fadhel\miniconda3\envs\kelas-rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
# embedding_model.encode(all_chunks_with_metadata[0]['text'])

In [13]:
import uuid
from qdrant_client import QdrantClient, models
from qdrant_client.models import VectorParams, Distance

In [14]:
vector_db = QdrantClient(host='localhost', port=6333)
collection_name = 'specs_collection'
vector_dim = embedding_model.get_sentence_embedding_dimension()
vector_dim

384

In [24]:
if not vector_db.collection_exists(collection_name):
    vector_db.create_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=vector_dim,
            distance=Distance.COSINE
        )
    )

In [20]:
all_texts = [item['text'] for item in all_chunks_with_metadata]
all_vectors = embedding_model.encode(all_texts, show_progress_bar=True)

points_to_upload = [
    models.PointStruct(
        id=str(uuid.uuid4()),
        vector=vector,
        payload={
            "text": all_chunks_with_metadata[i]['text'],
            "model_name": all_chunks_with_metadata[i]['model_name'],
            "source_document": all_chunks_with_metadata[i]['source_document']
        }
    )
    for i, vector in enumerate(all_vectors)
]

Batches: 100%|██████████| 3/3 [00:02<00:00,  1.33it/s]


In [26]:
points_to_upload[:5]

[PointStruct(id='0b06f6c5-955d-4228-bf0a-9315142e4416', vector=[-0.04925193265080452, 0.07462037354707718, -0.01991453394293785, -0.019688071683049202, -0.04539335519075394, -0.030803317204117775, -0.06199264153838158, 0.12424164265394211, -0.05313318595290184, -0.010114661417901516, 0.05078142508864403, -0.052003972232341766, 0.05258641764521599, -0.042306169867515564, -0.09373681247234344, -0.015016448684036732, -0.022067468613386154, -0.017697855830192566, -0.05350973829627037, -0.04421047866344452, 0.040660616010427475, 0.009476937353610992, 0.008084549568593502, -0.052462492138147354, -0.04144331440329552, 0.03617554157972336, -0.06418445706367493, 0.06413845717906952, 0.027498336508870125, -0.04559784755110741, -0.03455043584108353, 0.07430734485387802, -0.04832185059785843, -0.06792037189006805, 0.014262047596275806, -0.008005520328879356, -0.02756587229669094, -0.018727876245975494, 0.00043732658377848566, -0.007409574463963509, 0.0011646728962659836, -0.14114929735660553, 0.05

In [27]:
vector_db.upload_points(
    collection_name=collection_name,
    points=points_to_upload,
    wait=True
)